In [1]:
from sublimpy import utils, variables, tidy

import pandas as pd
import pytz
import altair as alt
alt.data_transformers.enable('json') # allows us to use altair with large datasets, side effect of this is that lots
                                     # of files are saved "altair-data-xxx.json". delete these at your leisure.

DataTransformerRegistry.enable('json')

# User inputs

In [2]:
# local files where you have downloaded/will download sos data
sos_download_dir='/data2/elilouis/sublimationofsnow/sosnoqc'

# dates for entire snow-on season
# start_date = '20221130'
# end_date = '20230509'

# sample dates
start_date = '20230210'
end_date = '20230213'

# Topics covered in this notebook
1. Introduce `sublimpy` tools for downloading (and cleaning) data, 
2. calculating additional variables,
3. and generating a "tidy" dataset.
4. Plotting tidy data with Altair

# Download data

using `utils.download_sos_data`, which:
1. downloads multiple daily files, handling download errors
2. merges them into a single dataframe,
3. Fills in missing data so the 5-minute interval time index is continuous

In [3]:

sos_ds = utils.download_sos_data(
    start_date,
    end_date,
    variable_names=variables.DEFAULT_VARIABLES,
    local_download_dir = sos_download_dir,
    cache = True,
    planar_fit = False # allows downloading of the planar-fitted data 
                       # provided by NCAR (does not exist after ~ March 12)
)

Caching...skipping download for 20230210
Caching...skipping download for 20230211
Caching...skipping download for 20230212
Caching...skipping download for 20230213


We also have a function for downloading the high rate data, which is only available for a few selected days right now

In [4]:
utils.download_sos_highrate_data_hour(
    date = '20221031', 
    hour = '00', # must specify an hour because these datasets are very large
)

'hr_noqc_geo/isfs_geo_hr_20221031_00.nc'

# Calculate additional useful variables

These functions get us a bunch of bonus variables, including other "versions" of temperature,
calculated using an estimate of height-adjusted pressure 

`['Tpot_10m_c', 'Tvirtual_10m_c', 'Tpotvirtual_10m_c']`

surface temperatures

`['Tsurf_c', 'Tsurf_rad_d']`

gradients,

`['temp_gradient_10m_c', 'wind_gradient_10m_c']`

Richardson numbers and Obukhov length

`['Ri_10m_c', 'RiB_10m_c', 'L_10m_c']`

Shear velocity, TKE

`['u*_10m_c', 'tke_10m_c']`

air density and mixing ratio

`['airdensity_10m_c', 'mixingratio_10m_c']`

and these exist for all heights at all towers where calculation is possible.

In [5]:
sos_ds = variables.add_surface_temps(sos_ds)
sos_ds = variables.add_potential_virtual_temperatures(sos_ds)
sos_ds = variables.add_surface_potential_virtual_temperatures(sos_ds)
sos_ds = variables.add_tke(sos_ds)
sos_ds = variables.add_gradients_and_ri(sos_ds)
sos_ds = variables.add_obukhov_length(sos_ds)

# Get Tidy Dataset

using `tidy.get_tidy_dataset` which converts the xarray/NetCDF data.

In [6]:
tidy_df = tidy.get_tidy_dataset(sos_ds, list(sos_ds.data_vars))

# Localize timestamps 

using the utility `utils.modify_df_timezone`. If you want to work with localized xarray data,
`utils.modify_xarray_timezone` does the trick.

In [7]:
tidy_df = utils.modify_df_timezone(
    tidy_df, 
    source_tz = pytz.UTC, 
    target_tz = pytz.timezone('US/Mountain')
)
tidy_df = tidy_df[[ 'time', 'tower', 'height', 'measurement', 'variable', 'value']]

# What is tidy data?

The simplest tidy_data would look like this:

In [8]:
tidy_df[['time', 'variable', 'value']]

,time,variable,value
0,2023-02-09 17:02:30,w_w__20m_c,0.067976
1,2023-02-09 17:07:30,w_w__20m_c,0.109061
2,2023-02-09 17:12:30,w_w__20m_c,0.071565
3,2023-02-09 17:17:30,w_w__20m_c,0.034470
4,2023-02-09 17:22:30,w_w__20m_c,0.031268
...,...,...,...
570235,2023-02-13 16:37:30,L_20m_c,21.407660
570236,2023-02-13 16:42:30,L_20m_c,64.408348
570237,2023-02-13 16:47:30,L_20m_c,52.224754
570238,2023-02-13 16:52:30,L_20m_c,-26.747652


Notice that the variable names actually contain 3 pieces of information - the type of measurement, the height, and the tower of that measurement.
We can split that information into separate columns so that our dataset allows easy indexing of what we want. This is performed by mapping each variable name to a "measurement" name, a height (0-20 meters, or negative for soil temperature measurements), and a tower (uw, ue, c, d).

This is exactly what `tidy.get_tidy_dataset` gives us.

In [9]:
tidy_df.sample(5)

,time,tower,height,measurement,variable,value
13942,2023-02-10 02:52:30,c,3.0,v_tc_,v_tc__3m_c,0.140419
110499,2023-02-13 09:17:30,c,10.0,u_w_,u_w__10m_c,-0.007126
549115,2023-02-12 08:37:30,c,20.0,richardson number,Ri_20m_c,3.793491
131259,2023-02-13 11:17:30,uw,10.0,v_v_,v_v__10m_uw,0.040363
533083,2023-02-12 16:37:30,c,10.0,wind gradient,wind_gradient_10m_c,0.023853


# Plot 

Tidy data can make working with time series data extremely easy, especially with the Altair plotting library.

## Snow-thermistor array time series

In [10]:
src = tidy_df.query("measurement == 'snow temperature'").query("tower == 'd'")

In [11]:
alt.Chart(src).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Temperature (˚C)"),
    alt.Color("height:O").scale(scheme='purpleorange')
).properties(width=450)

alt.Chart(...)

In [12]:
alt.Chart(src).mark_line().transform_window(
    rolling_mean = 'mean(value)',
    frame=[-6,6]
).encode(
    alt.X("time:T"),
    alt.Y("rolling_mean:Q").title(["Temperature (˚C)", "60min rolling average"]),
    alt.Color("height:O").scale(scheme='rainbow')
).properties(width=450)

alt.Chart(...)

## Profile time series

In [13]:
src = tidy_df.query("measurement == 'snow temperature'")

In [14]:
alt.Chart(src).mark_line().transform_window(
    rolling_mean = 'mean(value)',
    frame=[-6,6]
).encode(
    alt.X("time:T"),
    alt.Y("rolling_mean:Q").title("Temperature (˚C)"),
    alt.Color("height:O").scale(scheme='rainbow'),
    alt.Facet("tower:N")
).properties(width=450).configure_legend(orient='top')

alt.Chart(...)

In [15]:
src = tidy_df.copy().query("tower == 'c'")
src['hour'] = src['time'].dt.hour
src['day'] = src['time'].dt.day
src = src[src['time'].dt.minute < 5]
src = src[src['hour']%4 == 0]
src = src.query("day > 9").query("day < 13")

In [16]:
temp_prof_chart = alt.Chart(
    pd.concat([
        src.query("measurement == 'temperature'"),
        src.query("variable == 'Tsurf_c'")
    ])
).mark_line().encode(
    alt.X("value:Q").title('Temperature (˚C)').sort('-y'),
    alt.Y("height:Q"),
    alt.Color("hour:O").scale(scheme='turbo'),
    alt.Facet("day:O")
).properties(
    height = 100, 
    width=300
).resolve_scale(
    x='independent'
)
temp_prof_chart

alt.Chart(...)

In [17]:
wind_prof_chart = alt.Chart(src.query("measurement == 'wind speed'")).mark_line().encode(
    alt.X("value:Q").title('Wind speed (m/s)').sort('-y'),
    alt.Y("height:Q"),
    alt.Color("hour:O").scale(scheme='turbo'),
    alt.Facet("day:O")
).properties(
    height = 100, 
    width=300
).resolve_scale(
    x='independent'
)
wind_prof_chart

alt.Chart(...)

In [18]:
watervapor_prof_chart = alt.Chart(src.query("measurement == 'mixing ratio'")).mark_line().encode(
    alt.X("value:Q").title('Water vapor mixing ratio (g/g)').sort('-y'),
    alt.Y("height:Q"),
    alt.Color("hour:O").scale(scheme='turbo'),
    alt.Facet("day:O")
).properties(
    height = 100, 
    width=300
).resolve_scale(
    x='independent'
)
watervapor_prof_chart

alt.Chart(...)

In [19]:
wind_prof_chart & temp_prof_chart & watervapor_prof_chart

alt.VConcatChart(...)

## Scatterplots of Ri and turbulent fluxes

In [20]:
wide_df = tidy_df[tidy_df['variable'].isin(['Ri_3m_c', 'RiB_3m_c', 'w_h2o__3m_c', 'w_tc__3m_c'])].reset_index(drop=True).pivot(
    values = ['value'],
    columns = 'variable',
    index = 'time'
)
wide_df.columns = wide_df.columns.get_level_values(1)
wide_df = wide_df.reset_index()

In [21]:
rule = alt.Chart().mark_rule().encode(x=alt.datum(0.25))
xscale = alt.Scale(domain = [-2, 4], clamp=True)
yscale = alt.Scale(domain = [-.01, .01], clamp=True)

base = alt.Chart(wide_df)

points = base.mark_circle(size=2).encode(
    alt.X('Ri_3m_c').scale(xscale),
    alt.Y('w_h2o__3m_c').scale(yscale)
).properties(width=150, height=150)

top_hist = base.mark_bar().encode(
    alt.X("Ri_3m_c")
    .bin(maxbins=30, extent=xscale.domain).scale(clamp=True)
    .title(None)
    .axis(labels=False),
    alt.Y('count()').title(None)

).properties(width=150, height=30)

w_h2o_vs_ri_plot = top_hist & (points + rule)
ri_chart = w_h2o_vs_ri_plot

In [22]:
rule = alt.Chart().mark_rule().encode(x=alt.datum(0.2))
xscale = alt.Scale(domain = [-2, 4], clamp=True)
yscale = alt.Scale(domain = [-.01, .01], clamp=True)

base = alt.Chart(wide_df)

points = base.mark_circle(size=2).encode(
    alt.X('RiB_3m_c').scale(xscale),
    alt.Y('w_h2o__3m_c').scale(yscale)
).properties(width=150, height=150)

top_hist = base.mark_bar().encode(
    alt.X("RiB_3m_c")
    .bin(maxbins=30, extent=xscale.domain).scale(clamp=True)
    .title(None)
    .axis(labels=False),
    alt.Y('count()').title(None)
    
).properties(width=150, height=30)

right_hist = base.mark_bar().encode(
    alt.Y("w_h2o__3m_c")
    .bin(maxbins=30, extent=yscale.domain)
    .title(None)
    .axis(labels=False),
    alt.X('count()').title(None)
).properties(width=30, height = 150)

w_h2o_vs_ri_plot = top_hist & (points + rule | right_hist)
rib_chart = w_h2o_vs_ri_plot

In [23]:
(ri_chart | rib_chart)

alt.HConcatChart(...)